In [ ]:
import gzip
import json
import pandas as pd

from surprise import SVD, NMF
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split

from collections import defaultdict

Load data (user id, asin, rating)

In [ ]:
with gzip.open(r"F:\work\is590ml_final\data\Office_Products.json.gz", 'rt', encoding='utf-8') as f:
    data = []
    for line in f:
        review = json.loads(line)
        data.append((review['reviewerID'], review['asin'], review['overall']))

In [ ]:
office = pd.DataFrame(data, columns = ['user_id', 'asin', 'rating'])

drop users who have reviewd less than 5 times

In [ ]:
office = office.groupby('user_id').filter(lambda x: len(x) > 5)

In [ ]:
office.head()

drop products which have been reviewed less than 6 times

In [ ]:
office = office.groupby('asin').filter(lambda x: len(x) > 15)

In [ ]:
office.to_csv('office.csv')

Use surprise library to parse the data frame and perform cross validation to find a good matrix factorization method

In [ ]:
reader = Reader(rating_scale=(1, 5))
office_data = Dataset.load_from_df(office, reader=reader)

In [ ]:
algo = SVD()
cross_validate(algo, office_data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

In [ ]:
algo = NMF()
cross_validate(algo, office_data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

split the data into test and train datasets and train the model. Find the accuracy to report

In [ ]:
trainset, testset = train_test_split(office_data, test_size=0.05)
algo = SVD()
predictions = algo.fit(trainset).test(testset)
# accuracy.rmse(predictions)

Some manual testing

In [ ]:
unique_asins = set(office.asin.unique())
with gzip.open(r"F:\work\is590ml_final\data\meta_Office_Products.json.gz", 'rt', encoding='utf-8') as f:
    meta = {}
    for line in f:
        metadata = json.loads(line)
        if metadata['asin'] in unique_asins:
            meta[metadata['asin']] = metadata.get('title', '')

In [ ]:
user_pred = {}
for asin in meta:
    user_pred[asin] = algo.predict('A2I8NFMQROGJHR', asin).est

In [ ]:
max(user_pred, key=lambda x: user_pred[x])

In [ ]:
meta['B00006IF6W']

In [ ]:
user_pred = algo.predict(user_test)